In [1]:
import numpy as np
import pydot
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import sys
import cv2 as cv
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from subprocess import check_output
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
from subprocess import check_output

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [3]:
NUM_CLASSES = 5
# we need images of same size so we convert them into the size
WIDTH = 128
HEIGHT = 128
DEPTH = 3
inputShape = (HEIGHT, WIDTH, DEPTH)
# initialize number of epochs to train for, initial learning rate and batch size
EPOCHS = 32
INIT_LR = 1e-5
BS = 55
#global variables
ImageNameDataHash = {}
ImageNameDataHashTest = {}
uniquePatientIDList = []

In [4]:
def readTrainData(trainDir):
    global ImageNameDataHash
    # loop over the input images
    images = os.listdir(trainDir)
    print("Number of files in " + trainDir + " is " + str(len(images)))
    for imageFileName in images:
        if (imageFileName == "TrainLabels.csv"):
            continue
        print(imageFileName)
        # load the image, pre-process it, and store it in the data list
        imageFullPath = os.path.join(trainDir, imageFileName)
        #print(imageFullPath)
        print(imageFullPath)
        
        img = load_img(imageFullPath)
       
        arr = img_to_array(img)  # Numpy array with shape (233,233,3)
        dim1 = arr.shape[0]
        dim2 = arr.shape[1]
        dim3 = arr.shape[2]
 
        if (dim1 < HEIGHT or dim2 < WIDTH or dim3 < DEPTH):
            print("Error image dimensions are less than expected "+str(arr.shape))
        arr = cv.resize(arr, (HEIGHT,WIDTH)) #Numpy array with shape (HEIGHT, WIDTH,3)
        #print(arr.shape) # 128,128,3
        dim1 = arr.shape[0]
        dim2 = arr.shape[1]
        dim3 = arr.shape[2]
        if (dim1 != HEIGHT or dim2 != WIDTH or dim3 != DEPTH):
            print("Error after resize, image dimensions are not equal to expected "+str(arr.shape))
        #print(type(arr))
        # scale the raw pixel intensities to the range [0, 1] - TBD TEST
        
        arr = np.array(arr, dtype="float") / 255.0
        
        imageFileName = imageFileName.replace('.jpg','')
        print(imageFileName)
      
        ImageNameDataHash[str(imageFileName)] = np.array(arr) 
        
    return

In [5]:
def readTestData(testDir):
    global ImageNameDataHashTest
    # loop over the input images
    images = os.listdir(testDir)
    print("Number of files in " + testDir + " is " + str(len(images)))
    for imageFileName in images:
        if (imageFileName == "TestLabels.csv"):
            continue
        print(imageFileName)
        # load the image, pre-process it, and store it in the data list
        imageFullPath = os.path.join(testDir, imageFileName)
        #print(imageFullPath)
        print(imageFullPath)
        
        img = load_img(imageFullPath)
       
        arr = img_to_array(img)  # Numpy array with shape (233,233,3)
        dim1 = arr.shape[0]
        dim2 = arr.shape[1]
        dim3 = arr.shape[2]
 
        if (dim1 < HEIGHT or dim2 < WIDTH or dim3 < DEPTH):
            print("Error image dimensions are less than expected "+str(arr.shape))
        arr = cv.resize(arr, (HEIGHT,WIDTH)) #Numpy array with shape (HEIGHT, WIDTH,3)
        #print(arr.shape) # 128,128,3
        dim1 = arr.shape[0]
        dim2 = arr.shape[1]
        dim3 = arr.shape[2]
        if (dim1 != HEIGHT or dim2 != WIDTH or dim3 != DEPTH):
            print("Error after resize, image dimensions are not equal to expected "+str(arr.shape))
        #print(type(arr))
        # scale the raw pixel intensities to the range [0, 1] - TBD TEST
        
        arr = np.array(arr, dtype="float") / 255.0
        
        imageFileName = imageFileName.replace('.jpg','')
        print(imageFileName)
         
        
        ImageNameDataHashTest[str(imageFileName)] = np.array(arr) 
        
    return

In [6]:
from datetime import datetime
print("Loading images at..."+ str(datetime.now()))
sys.stdout.flush()
readTrainData('B. Disease Grading/1. Original-Images/TrainSet')
print("Loaded " + str(len(ImageNameDataHash)) + " images at..."+ str(datetime.now())) # 1000

Loading images at...2018-12-09 16:37:09.191501
Number of files in B. Disease Grading/1. Original-Images/TrainSet is 414
IDRiD_060.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_060.jpg
IDRiD_060
IDRiD_345.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_345.jpg
IDRiD_345
IDRiD_073.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_073.jpg
IDRiD_073
IDRiD_130.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_130.jpg
IDRiD_130
IDRiD_333.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_333.jpg
IDRiD_333
IDRiD_387.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_387.jpg
IDRiD_387
IDRiD_342.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_342.jpg
IDRiD_342
IDRiD_105.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_105.jpg
IDRiD_105
IDRiD_178.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_178.jpg
IDRiD_178
IDRiD_272.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_272.jpg
IDRiD_272
IDRiD_089.jpg
B. Disease Gradi

IDRiD_192
IDRiD_234.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_234.jpg
IDRiD_234
IDRiD_141.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_141.jpg
IDRiD_141
IDRiD_171.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_171.jpg
IDRiD_171
IDRiD_076.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_076.jpg
IDRiD_076
IDRiD_215.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_215.jpg
IDRiD_215
IDRiD_010.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_010.jpg
IDRiD_010
IDRiD_041.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_041.jpg
IDRiD_041
IDRiD_297.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_297.jpg
IDRiD_297
IDRiD_357.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_357.jpg
IDRiD_357
IDRiD_359.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_359.jpg
IDRiD_359
IDRiD_275.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_275.jpg
IDRiD_275
IDRiD_242.jpg
B. Disease Grading/1. Original-Images/Tra

IDRiD_193
IDRiD_230.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_230.jpg
IDRiD_230
IDRiD_208.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_208.jpg
IDRiD_208
IDRiD_278.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_278.jpg
IDRiD_278
IDRiD_386.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_386.jpg
IDRiD_386
IDRiD_351.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_351.jpg
IDRiD_351
IDRiD_309.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_309.jpg
IDRiD_309
IDRiD_256.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_256.jpg
IDRiD_256
IDRiD_083.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_083.jpg
IDRiD_083
IDRiD_184.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_184.jpg
IDRiD_184
IDRiD_187.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_187.jpg
IDRiD_187
IDRiD_129.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_129.jpg
IDRiD_129
IDRiD_135.jpg
B. Disease Grading/1. Original-Images/Tra

IDRiD_036
IDRiD_086.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_086.jpg
IDRiD_086
IDRiD_282.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_282.jpg
IDRiD_282
IDRiD_163.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_163.jpg
IDRiD_163
IDRiD_371.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_371.jpg
IDRiD_371
IDRiD_301.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_301.jpg
IDRiD_301
IDRiD_395.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_395.jpg
IDRiD_395
IDRiD_017.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_017.jpg
IDRiD_017
IDRiD_375.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_375.jpg
IDRiD_375
IDRiD_325.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_325.jpg
IDRiD_325
IDRiD_149.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_149.jpg
IDRiD_149
IDRiD_095.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_095.jpg
IDRiD_095
IDRiD_070.jpg
B. Disease Grading/1. Original-Images/Tra

IDRiD_180
IDRiD_077.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_077.jpg
IDRiD_077
IDRiD_167.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_167.jpg
IDRiD_167
IDRiD_216.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_216.jpg
IDRiD_216
IDRiD_203.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_203.jpg
IDRiD_203
IDRiD_071.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_071.jpg
IDRiD_071
IDRiD_218.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_218.jpg
IDRiD_218
IDRiD_206.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_206.jpg
IDRiD_206
IDRiD_063.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_063.jpg
IDRiD_063
IDRiD_207.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_207.jpg
IDRiD_207
IDRiD_019.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_019.jpg
IDRiD_019
IDRiD_245.jpg
B. Disease Grading/1. Original-Images/TrainSet/IDRiD_245.jpg
IDRiD_245
IDRiD_069.jpg
B. Disease Grading/1. Original-Images/Tra

In [7]:
len(ImageNameDataHash)

413

In [8]:
from datetime import datetime
print("Loading images at..."+ str(datetime.now()))
sys.stdout.flush()
readTestData('B. Disease Grading/1. Original-Images/TestSet')
print("Loaded " + str(len(ImageNameDataHashTest)) + " images at..."+ str(datetime.now())) # 1000

Loading images at...2018-12-09 16:38:07.398455
Number of files in B. Disease Grading/1. Original-Images/TestSet is 104
IDRiD_060.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_060.jpg
IDRiD_060
IDRiD_073.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_073.jpg
IDRiD_073
IDRiD_089.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_089.jpg
IDRiD_089
IDRiD_031.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_031.jpg
IDRiD_031
IDRiD_046.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_046.jpg
IDRiD_046
IDRiD_102.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_102.jpg
IDRiD_102
IDRiD_072.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_072.jpg
IDRiD_072
IDRiD_082.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_082.jpg
IDRiD_082
IDRiD_054.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_054.jpg
IDRiD_054
IDRiD_015.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_015.jpg
IDRiD_015
IDRiD_055.jpg
B. Disease Grading/1. Origi

IDRiD_071
IDRiD_063.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_063.jpg
IDRiD_063
IDRiD_019.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_019.jpg
IDRiD_019
IDRiD_069.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_069.jpg
IDRiD_069
IDRiD_048.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_048.jpg
IDRiD_048
IDRiD_065.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_065.jpg
IDRiD_065
IDRiD_100.jpg
B. Disease Grading/1. Original-Images/TestSet/IDRiD_100.jpg
IDRiD_100
Loaded 103 images at...2018-12-09 16:38:21.825887


In [9]:
len(ImageNameDataHashTest)

103

In [10]:
import csv
def readTrainCsv():
    raw_df = pd.read_csv('B. Disease Grading/1. Original-Images/TrainSet/TrainLabels.csv', sep=',')
    print(type(raw_df)) #<class 'pandas.core.frame.DataFrame'>
    row_count=raw_df.shape[0] #gives number of row count row_count=35126 
    col_count=raw_df.shape[1] #gives number of col count col count=2
    print("row_count="+str(row_count)+" col count="+str(col_count))
    raw_df["PatientID"] = ''
    header_list = list(raw_df.columns)
    print(header_list) # ['image', 'level', 'PatientID']
    # double check if level of left and right are same or not
    ImageLevelHash = {}
    patientIDList = []
    for index, row in raw_df.iterrows():
        # 0 is image, 1 is level, 2 is PatientID, 3 is data
        key = row[0] + ''
        patientID = row[0] + ''
        patientID = patientID.replace('_right','')
        patientID = patientID.replace('_left','')
        #print("Adding patient ID"+ patientID)
        raw_df.at[index, 'PatientID'] = patientID
        patientIDList.append(patientID)
        ImageLevelHash[key] = str(row[1]) # level
                
    global uniquePatientIDList
    uniquePatientIDList = sorted(set(patientIDList))
    count=0;
    for patientID in uniquePatientIDList:
        left_level = ImageLevelHash[str(patientID+'_left')]
        right_level = ImageLevelHash[str(patientID+'_right')]
        #right_exists = str(patientID+'_right') in raw_df.values
        if (left_level != right_level):
            count = count+1
            #print("Warning for patient="+ str(patientID) + " left_level=" + left_level+ " right_level=" +right_level)
    print("count of images with both left and right eye level not matching="+str(count)) # 2240
    print("number of unique patients="+str(len(uniquePatientIDList))) # 17563
    return raw_df

In [11]:
raw_df = pd.read_csv('B. Disease Grading/1. Original-Images/TrainSet/TrainLabels.csv', sep=',',skip_blank_lines=True,date_parser="Image name")
raw_df=raw_df.loc[:, ~raw_df.columns.str.contains('^Unnamed')]
raw_df_test=pd.read_csv('B. Disease Grading/1. Original-Images/TestSet/TestLabels.csv', sep=',',skip_blank_lines=True,date_parser="Image name")
raw_df_test=raw_df_test.loc[:, ~raw_df_test.columns.str.contains('^Unnamed')]
rawddf_num = np.array(raw_df)
#rawddf_num[:,:1]
#print(len(rawddf_num[:,:1]))
#rawddf_num[:,1:]
raw_df_test_num=np.array(raw_df_test)
raw_df_test_num[0][1]

4

In [12]:
header_list = list(raw_df.columns)
header_list

['Image name', 'Retinopathy grade', 'Risk of macular edema ']

In [13]:
imageNameArr = []
dataArr = []
rgradearr= []

num=len(rawddf_num[:,:1])
num=num
for index, row in raw_df.iterrows():
    key = str(row[0])
    if key in ImageNameDataHash:
        imageNameArr.append(key)
        dataArr.append(np.array(ImageNameDataHash[key])) # np.array
for i in range(num) :
    rgradearr.append(rawddf_num[i][1])
    i=i+1

df2 = pd.DataFrame({'Image_name': imageNameArr, 'data': dataArr, 'Retinopathy Grade': rgradearr})
df2_header_list = list(df2.columns) 
print(df2_header_list) # ['image', 'data']
print(len(df2)) # 1000


['Image_name', 'data', 'Retinopathy Grade']
413


In [14]:
imageNameArrtest=[]
dataArrtest=[]
rgradearrtest=[]

numtest=len(raw_df_test_num[:,:1])
for index, row in raw_df_test.iterrows():
    keyt=str(row[0])
    if keyt in ImageNameDataHashTest:
        imageNameArrtest.append(keyt)
        dataArrtest.append(np.array(ImageNameDataHashTest[keyt]))
for it in range(numtest) :
    rgradearrtest.append(raw_df_test_num[it][1])
    it=it+1  
df2test = pd.DataFrame({'Image_name': imageNameArrtest, 'data_test': dataArrtest, 'Retinopathy Grade_test': rgradearrtest})
df2test_header_list = list(df2test.columns) 
print(df2test_header_list) # ['image', 'data']
print(len(df2test)) # 1000


['Image_name', 'data_test', 'Retinopathy Grade_test']
103


In [15]:
df2test_num=np.array(df2test)
Y = df2['Retinopathy Grade']


In [16]:
X = df2['data']
Y = df2['Retinopathy Grade']
Y = np.array(Y)
Y =  to_categorical(Y, num_classes=NUM_CLASSES)
Y

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [17]:
print("Parttition data into 80:20...")
sys.stdout.flush()
unique_ids = df2.Image_name.unique()
unique_ids
unique_idstest = df2test.Image_name.unique()
#train_ids, valid_ids = train_test_split(unique_ids, test_size = 0.3, random_state = 10)
train_ids=unique_ids
valid_ids=unique_idstest
trainid_list = train_ids.tolist()
validid_list = valid_ids.tolist()
print('trainid_list shape=', str(len(trainid_list))) # 375
print('testid_list shape=', str(len(validid_list)))
traindf = df2[df2.Image_name.isin(trainid_list)]
valSet = df2test[df2test.Image_name.isin(validid_list)]

Parttition data into 80:20...
trainid_list shape= 413
testid_list shape= 103


In [18]:


print(traindf.head())
print(valSet.head())

traindf = traindf.reset_index(drop=True)
valSet = valSet.reset_index(drop=True)

print(traindf.head())
print(valSet.head())



  Image_name                                               data  \
0  IDRiD_001  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
1  IDRiD_002  [[[0.00784313725490196, 0.00784313725490196, 0...   
2  IDRiD_003  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
3  IDRiD_004  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0117647...   
4  IDRiD_005  [[[0.0, 0.0, 0.0], [0.0196078431372549, 0.0039...   

   Retinopathy Grade  
0                  3  
1                  3  
2                  2  
3                  3  
4                  4  
  Image_name                                          data_test  \
0  IDRiD_001  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
1  IDRiD_002  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
2  IDRiD_003  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
3  IDRiD_004  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
4  IDRiD_005  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   

   Retinopathy Grade_test  
0                       4  


In [19]:
trainX = traindf['data']
trainY = traindf['Retinopathy Grade']

valX = valSet['data_test']
valY = valSet['Retinopathy Grade_test']

#(trainX, valX, trainY, valY) = train_test_split(X,Y,test_size=0.20, random_state=10)
print('trainX shape=', trainX.shape[0], 'valX shape=', valX.shape[0])

trainX shape= 413 valX shape= 103


In [20]:
trainY =  to_categorical(trainY, num_classes=NUM_CLASSES)
valY =  to_categorical(valY, num_classes=NUM_CLASSES)

In [21]:
print("Generating images...")
sys.stdout.flush()
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, \
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,\
    horizontal_flip=True, fill_mode="nearest")

Generating images...


In [35]:
def createModel():
    model = Sequential()
    # first set of CONV => RELU => MAX POOL layers
    model.add(Conv2D(32, (2, 2), padding='same', activation='relu', input_shape=inputShape))
    model.add(Conv2D(32, (2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    
    model.add(Conv2D(64, (4, 4), padding='same', activation='relu'))
    model.add(Conv2D(64, (4, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (4, 4), padding='same', activation='relu'))
    model.add(Conv2D(64, (4, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_dim=NUM_CLASSES, activation='softmax'))
    # returns our fully constructed deep learning + Keras image classifier 
    opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    # use binary_crossentropy if there are two classes
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model


In [36]:
print("Reshaping trainX at..."+ str(datetime.now()))
#print(trainX.sample()) 
print(type(trainX)) # <class 'pandas.core.series.Series'>
print(trainX.shape) # (750,)
from numpy import zeros
Xtrain = np.zeros([trainX.shape[0],HEIGHT, WIDTH, DEPTH])
for i in range(trainX.shape[0]): # 0 to traindf Size -1
    Xtrain[i] = trainX[i]
print(Xtrain.shape) # (750,128,128,3)
print("Reshaped trainX at..."+ str(datetime.now()))

Reshaping trainX at...2018-12-09 16:58:49.330668
<class 'pandas.core.series.Series'>
(413,)
(413, 128, 128, 3)
Reshaped trainX at...2018-12-09 16:58:49.419357


In [37]:
print("Reshaping valX at..."+ str(datetime.now()))
print(type(valX)) # <class 'pandas.core.series.Series'>
print(valX.shape) # (250,)
from numpy import zeros
Xval = np.zeros([valX.shape[0],HEIGHT, WIDTH, DEPTH])
for i in range(valX.shape[0]): # 0 to traindf Size -1
    Xval[i] = valX[i]
print(Xval.shape) # (250,128,128,3)
print("Reshaped valX at..."+ str(datetime.now()))

Reshaping valX at...2018-12-09 16:58:49.762430
<class 'pandas.core.series.Series'>
(103,)
(103, 128, 128, 3)
Reshaped valX at...2018-12-09 16:58:49.782120


In [38]:
Xtrain.shape

(413, 128, 128, 3)

In [39]:
print("compiling model...")
sys.stdout.flush()
model = createModel()

# print the summary of model
from keras.utils import print_summary
print_summary(model, line_length=None, positions=None, print_fn=None)

# add some visualization
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
SVG(model_to_dot(model).create(prog='dot', format='svg'))

compiling model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 128, 128, 32)      416       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 127, 127, 32)      4128      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 63, 63, 64)        32832     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 60, 60, 64)        65600     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 30, 30, 64)        0 

/home/nemathahmed/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=5)`


OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

In [41]:
# train the network
print("training network...")
sys.stdout.flush()
#class_mode ='categorical', # 2D one-hot encoded labels
H = model.fit_generator(aug.flow(Xtrain, trainY, batch_size=BS), \
    validation_data=(Xval, valY), \
    steps_per_epoch=len(trainX) // BS, \
    epochs=EPOCHS, verbose=1) 

# save the model to disk
print("Saving model to disk")
sys.stdout.flush()
model.save("/tmp/mymodel")

training network...
Epoch 1/7


KeyboardInterrupt: 

In [ ]:
print("Generating plots...")
import matplotlib
sys.stdout.flush()
matplotlib.use("Agg")
matplotlib.pyplot.style.use("ggplot")
matplotlib.pyplot.figure()
N = EPOCHS
matplotlib.pyplot.plot(np.arange(0, N), H.history["loss"], label="train_loss")
matplotlib.pyplot.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
matplotlib.pyplot.plot(np.arange(0, N), H.history["acc"], label="train_acc")
matplotlib.pyplot.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
matplotlib.pyplot.title("Training Loss and Accuracy on diabetic retinopathy detection")
matplotlib.pyplot.xlabel("Epoch #")
matplotlib.pyplot.ylabel("Loss/Accuracy")
matplotlib.pyplot.legend(loc="lower left")
matplotlib.pyplot.savefig("plot.png")

In [ ]:
from keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
# define model
model.fit(trainX, trainY,
          batch_size=BS,
          epochs=EPOCHS,
          validation_data=(valX, valY),
          shuffle=True,
          callbacks=[tensorboard])